In [17]:
from mp_api.client import MPRester
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

API_KEY = "4IeHY5jVcrgiKXNuAo6Jgs7yC0Z3hsli"

with MPRester(API_KEY) as mpr:
    # Pull summary data
    materials = mpr.materials.insertion_electrodes.search(
            working_ion="Li",
            fields=[
                "battery_id", "formula_discharge", "average_voltage"
                , "energy_grav", "energy_vol", "capacity_grav", "capacity_vol", "stability_charge", "fracA_charge", "stability_charge", "max_delta_volume", "stability_discharge", "fracA_discharge"
            ]
        )

# Convert to DataFrame
mp_df = pd.DataFrame([{
    "battery_id": m.material_id,
    "formula_discharge": m.formula_pretty,
    "band_gap": m.band_gap,
    "formation_energy": m.formation_energy_per_atom,
    "energy_above_hull": m.energy_above_hull,
    "density": m.density
} for m in materials])

# Drop any rows with missing values (critical)
mp_df = mp_df.dropna().reset_index(drop=True)

# Verify data exists
print("Number of materials pulled:", len(mp_df))
if len(mp_df) == 0:
    raise ValueError("No materials data available after dropping missing values!")

# -------------------------------
# Normalize and prepare features
# -------------------------------
features = ["formation_energy", "energy_above_hull", "density", "band_gap"]
scaler = MinMaxScaler()
mp_df[features] = scaler.fit_transform(mp_df[features])

# Invert features where lower is better
for col in ["formation_energy", "energy_above_hull", "band_gap"]:
    mp_df[col] = 1 - mp_df[col]

# Compute composite intrinsic score
mp_df["intrinsic_score"] = mp_df[features].mean(axis=1)

# -------------------------------
# Rank and select top candidates
# -------------------------------
top_candidates = mp_df.sort_values("intrinsic_score", ascending=False).head(20)

# Print to check
print("Top candidates based on intrinsic score:")
print(top_candidates[["material_id", "formula", "intrinsic_score"]])



Retrieving SummaryDoc documents: 100%|██████████| 9/9 [00:00<00:00, 136277.03it/s]

Number of materials pulled: 9
Top candidates based on intrinsic score:
  material_id formula  intrinsic_score
0    mp-10173      Li         0.988564
6   mp-567337      Li         0.967434
5       mp-51      Li         0.963726
1  mp-1018134      Li         0.959582
8   mp-976411      Li         0.953175
4      mp-135      Li         0.949252
2  mp-1063005      Li         0.926681
3  mp-1103107      Li         0.879235
7   mp-604313      Li         0.250000
